# Example 1 - Building a Layered Dike

This example defines a digital twin simulation in which several layers (from different materials) of a dike are constructed.

## Necessary Dependencies

In [1]:
# package(s) related to time, space and id
import datetime
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model

## Create the necessary classes

The Digital Twin core defines several classes which can be combined in mixin classes you can define yourself. For example, in this simulation we define a Site as:
* Identifiable: an object that can be identified with a name 
* Log: an object which will contain a log of events that took place concerning this object after the simulation is complete
* Locatable: an object that has a geographical location
* HasContainer: an object which contains a storage for materials
* HasResource: an object only a limited number of activities can interact with at the same time

In [2]:
Site = type('Site', (core.Identifiable, core.Log, core.Locatable, core.HasContainer, core.HasResource), {})

Defining mixin classes for our ships and cranes is done similarly. A ship can either be a simple transport barge we will call a TransportResource, or can be a hopper capable of loading and unloading the materials itself (TransportProcessingResource). Finally we define a class for "crane" objects which can be used to load or unload material for a transport barge (ProcessingResource).

In [3]:
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                          core.HasResource), {})

TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                                    core.Processor, core.HasResource), {})
ProcessingResource = type('ProcessingResource', 
                          (core.Identifiable, core.Log, core.Processor, core.HasResource), {})

These mixin classes use a couple of core classes not yet introduced:
* Processor: an object which can move materials from one container to another container
* ContainerDependentMovable: this is in fact a combination (extension) of two other core classes:
    * HasContainer: an object which contains a storage for materials
    * Movable: an object which can move from one geographical location to another (extends Locatable, clearly a moving object must keep track of its current location)
  
Defining a mixin class as a ContainerDependentMovable allows for specifying a speed function which depends on the fullness of the storage container. The more basic Movable class uses a fixed speed instead.




## Defining a Simpy Environment

Since the digital twin package uses the Simpy package for its backend, each object we create will need to be passed the same Simpy environment object. This Simpy environment is used during the simulation. In fact, it can be used to start the simulation later on. For now, we need to create the environment object, so we can pass it to each of our mixin objects when we create them. 


In [4]:
my_env = simpy.Environment()



## Defining the Sites

For our simulation we need to define Sites for the different material stockpiles, and sites along the length of the dike in which the material needs to be installed. This example includes four material layers: clay, sand, armour and levvel. We need to define a stockpile site for each of these materials. We will store each of the sites defined in this section in a "stocks" list to be used later.


In [5]:
# sites data in database
data_stock_clay = {"env": my_env,
                   "name": "Clay_Stock",
                   "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150000,
                   "level": 150000}
data_stock_sand = {"env": my_env,
                   "name": "Sand_Stock",
                   "geometry": shapely.geometry.Point(5.271417603333632, 52.9638452897506),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}
data_stock_armour = {"env": my_env,
                   "name": "Armour_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}
data_stock_levvel = {"env": my_env,
                   "name": "Levvel_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

# create the site objects
stocks = []
for data in [data_stock_clay, data_stock_sand, data_stock_armour, data_stock_levvel]:
    stock = Site(**data)
    stocks.append(stock)

The constructor parameters needed will differ depending on the core digital twin classes used in the mixin class you've defined. We can best explain each of the parameters we pass in turn:
* __env__: Most core digital twin classes extend a class called SimpyObject and need an "env" construction parameter. These classes, such as HasContainer and HasResource, need you to pass the Simpy environment so they can create Simpy objects within the simulation environment. 
* __name__: An Identifiable object needs a name, which will be used to identify the object.
* __geometry__: A Locatable object needs a geographical location. The object will initially be located at this location. In the case of a Movable object, the geographical location will be updated as it moves. If the object is not movable, the geometry should remain the same throughout the simulation.
* __capacity__: A HasContainer object needs to know the maximum amount of materials it can store.
* __level__: A HasContainer object can optionally be passed the initial level of the container. If no level is passed, the container is initially empty. In the case of our material stockpiles, we want them to be filled instead. 

We could also have passed a __nr_resources__ argument to the constructors to define the number of activities which can use one of these stockpiles at the same time. By default this is set to one, which means only one ship can be loaded at each stockpile at the same time during the simulation.

In addition to the stockpiles, we also need to define the destination sites in which the material will be placed. To do so we will define several sites spaced evenly along the length of the dike for each of the layers. 

In [6]:
site_count = 20

# layer data from database
data_clay = {"start": [5.054676856441372, 52.94042293840172], # Den Oever (lon, lat)
             "stop": [5.294877712236641, 53.06686424241725], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Clay_Layer"}
data_sand = {"start": [5.052051052879287,52.9421894472733], # Den Oever (lon, lat)
             "stop": [5.292216781509101,53.06886359869087], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Sand_Layer"}
data_armour = {"start": [5.049510554598302,52.94393628899332], # Den Oever (lon, lat)
             "stop": [5.289636346490858,53.07053144816584], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Armour_Layer"}
data_levvel = {"start": [5.046556507026805,52.94579445406793], # Den Oever (lon, lat)
             "stop": [5.286775240694118,53.07264015015531], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Levvel_Layer"}

sites = []
for data_layer in [data_clay, data_sand, data_armour, data_levvel]:
    start = data_layer['start']
    stop = data_layer['stop']

    # generate the evenly spaced locations between the selected start and stop points
    lons = np.linspace(start[0], stop[0], num=site_count)
    lats = np.linspace(start[1], stop[1], num=site_count)

    # create a Site object for each location
    layer_sites = []
    for i in range(site_count):
        data_site = {"env": my_env,
                     "name": data_layer["name"] + '_' + format(i, '02.0f'),
                     "geometry": shapely.geometry.Point(lons[i], lats[i]),
                     "capacity": data_layer["capacity"]}
        site = Site(**data_site)
        layer_sites.append(site)
    sites.append(layer_sites)

## Defining the Equipment

We need to create objects for the equipment used in the simulation. This is done similarly to creating the sites.

In [7]:
# define a simple linear function with which the speed of a ship can be calculated
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

# equipment information in database
data_gantry_crane = {"env": my_env,
                     "name": "Gantry crane",
                     "rate": 0.10}
data_installation_crane = {"env": my_env,
                           "name": "Installation crane",
                           "rate": 0.05}

data_transport_barge_01 = {"env": my_env,
                           "name": "Transport barge 01",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}
data_transport_barge_02 = {"env": my_env,
                           "name": "Transport barge 02",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

data_hopper = {"env": my_env,
               "name": "Hopper",
               "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),  # lon, lat
               "rate": 2,
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=2, v_full=1.5)}

# create the processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# create the transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# create the transport processing resource
hopper = TransportProcessingResource(**data_hopper)

To define our equipment we needed two new constructor arguments:
* __rate__: The speed with which a Processor can process materials
* __compute_v__: A function which given a value in the interval [0,1], representing the fullness of the container, returns the speed of the ContainerDependentMovable

## Defining the Activities

To complete the simulation we need to define which activities need to be completed to complete the simulation. An activity always consists of transporting materials from an origin container to a destination container. We need to define an activity for each origin, destination and the equipment used to transport the materials. For installing the clay layer we will use the transport barges, gantry and installation crane. Since we have two transport barges that could be used, we define two activities for each of the clay layer sites, one for each set of equipment. We also wish to install the clay layer in order, so for each site we need to define a condition that states that the previous site needs to be completed first.

In [8]:
activities = []

clay_layer_index = 0
clay_sites = sites[clay_layer_index]
for i, clay_site in enumerate(clay_sites):
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        # for the first site we do not need to define any condition
        condition = None if i == 0 else model.LevelCondition(container=clay_sites[i-1], min_level=data_clay["capacity"])
        activity = model.Activity(env=my_env,
                                  name="Clay_Placement_S" + format(i+1, '02.0f') + "_B" + format(barge_index + 1, '02.0f'),
                                  origin=stocks[clay_layer_index],
                                  destination=clay_site,
                                  loader=gantry_crane,
                                  mover=transport_barge,
                                  unloader=installation_crane,
                                  start_condition=condition)
        activities.append(activity)

As can be seen in the code fragment above, the Activity class takes the Simpy environment and a name as its constructor arguments. This is because it is defined as a mixin of the SimpyObject and Identifiable core classes. To construct an activity we also needed to define the origin, destination, loader, mover and unloader. Finally, we can optionally define conditions that need to be satisfied for the transportation of materials to take place. These are separated in __start_condition__, __stop_condition__ and __condition__ arguments:
* __start_condition__: The activity will start as soon as this condition is satisfied. If this condition is not yet satisfied at the start of the simulation the activity will be suspended untill the condition is satisfied at a later time. Since we do not want any clay to be transported to the later sites if the previous site has not yet been completed, we define this condition as a start_condition.
* __stop_condition__: After the activity has started (the start_condition was satisfied), it will regularly check if the stop_condition has been satisfied. If so, the activity is considered complete and will terminate. A simulation will not terminate untill all of its activities have been completed. By default (with stop_condition=None), the stop_condition is set to either the origin container being empty, or the destination container being full. This is sufficient in our case, so we do not specify a stop_condition. 
* __condition__: After the activity has started (the start_condition was satisfied) and as long as the stop_condition is not satisfied, the activity will continuously check whether this condition is satisfied. If so, it will complete exactly one transportation of materials (loading the mover at the origin and unloading the mover at the destination). If not, the activity will wait untill the condition is satisfied again. Defining such a condition allows for the activity to be temporarily halted, for example due to bad weather or other working conditions, and resuming at a later time when the problem preventing work has passed. 

All of these condition arguments take a "Condition" object. This object should have a "satisfied" method which returns a boolean indicating whether the condition is satsified or not. The digital twin package provides a LevelCondition, allowing you to quickly construct a condition which will check if an object extending HasContainer has a certain minimum and/or maximum level. It also provides an AndCondition and OrCondition which allow for combining other condition objects into a single condition. You can also provide your own "Condition" class, for example you could program a "Condition" class which depending on the time in the simulation (the .now fixture of the Simpy environment), returns whether the weather is good enough for work to take place:

In [9]:
class WeatherCondition:
    def __init__(self, env, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
    
    def satisfied(self):
        return self.env.now < 30_000 or self.env.now > 40_000  # between time step 30_000 and 40_000 a "storm" takes place

For our simple dike building example we do not define any advanced conditions. We do need to define Activities for the other layers (sand, armour and levvel). For each of these layers we again require that they are installed in order. We also require the previous layer to be completed, before the next layer can be installed. To achieve this we combine two LevelConditions with an AndCondition. We choose to install the sand layer using the hopper, and the armour and levvel layers using the transport barges and grantry and installation cranes.

In [10]:
# define the sand layer Activities
sand_layer_index = 1
sand_sites = sites[sand_layer_index]
for i, sand_site in enumerate(sand_sites):
    # for the first site we do not need to define a condition on the previous site
    prev_site_condition = model.TrueCondition() if i == 0 else model.LevelCondition(container=sand_sites[i-1], 
                                                                          min_level=sand_sites[i-1].container.capacity)
    prev_layer_sites = sites[clay_layer_index]
    prev_layer_condition = model.LevelCondition(prev_layer_sites[i], min_level=prev_layer_sites[i].container.capacity)
    start_condition = model.AndCondition([prev_site_condition, prev_layer_condition])
    activity = model.Activity(env=my_env,
                              name="Sand_Placement_S" + format(i+1, '02.0f'),
                              origin=stocks[sand_layer_index],
                              destination=sand_site,
                              loader=hopper,
                              mover=hopper,
                              unloader=hopper,
                              start_condition=start_condition)
    activities.append(activity)

# define the armour and level layer activities
layer_names = ["Armour_Placement", "Levvel_Placement"]
layer_indices = [2, 3]
for layer_index in layer_indices:
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        layer_sites = sites[layer_index]
        for site_index, site in enumerate(layer_sites):
            # for the first site we do not need to define a condition on the previous site
            prev_site_condition = model.TrueCondition() if site_index == 0 else \
                model.LevelCondition(container=layer_sites[site_index-1], 
                                     min_level=layer_sites[site_index-1].container.capacity)
            
            prev_layer_sites = sites[layer_index - 1]
            prev_layer_condition = model.LevelCondition(container=prev_layer_sites[site_index],
                                                        min_level=prev_layer_sites[site_index].container.capacity)
            start_condition = model.AndCondition([prev_site_condition, prev_layer_condition])
            
            activity = model.Activity(env=my_env,
                                      name=layer_names[layer_index-2] + "_S" + format(site_index+1, '02.0f')
                                           + "_B" + format(barge_index+1, '02.0f'),
                                      origin=stocks[layer_index],
                                      destination=site,
                                      loader=gantry_crane,
                                      mover=transport_barge,
                                      unloader=installation_crane,
                                      start_condition=start_condition)
            activities.append(activity)

## Running the Simulation

To run the simulation, we now simply need to tell the Simpy environment to run. This will run all activities (most of which will first suspend themselved unstill the start_condition is satisfied) and terminate once every Activity has reached its stop_condition. To allow for the visualisation later, we record the start time in the environment.

In [11]:
my_env.epoch = datetime.datetime.now()

my_env.run()

print("\n*** Installation of dike finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now))))

T=000.00 Start condition is satisfied, Clay_Placement_S01_B01 transporting from Clay_Stock to Clay_Layer_00 started
Using Transport barge 01 to process 1000
T=000.00 Start condition is satisfied, Clay_Placement_S01_B02 transporting from Clay_Stock to Clay_Layer_00 started
Using Transport barge 02 to process 1000
T=000.00 Clay_Placement_S02_B01 to Clay_Layer_01 suspended
T=000.00 Clay_Placement_S02_B02 to Clay_Layer_01 suspended
T=000.00 Clay_Placement_S03_B01 to Clay_Layer_02 suspended
T=000.00 Clay_Placement_S03_B02 to Clay_Layer_02 suspended
T=000.00 Clay_Placement_S04_B01 to Clay_Layer_03 suspended
T=000.00 Clay_Placement_S04_B02 to Clay_Layer_03 suspended
T=000.00 Clay_Placement_S05_B01 to Clay_Layer_04 suspended
T=000.00 Clay_Placement_S05_B02 to Clay_Layer_04 suspended
T=000.00 Clay_Placement_S06_B01 to Clay_Layer_05 suspended
T=000.00 Clay_Placement_S06_B02 to Clay_Layer_05 suspended
T=000.00 Clay_Placement_S07_B01 to Clay_Layer_06 suspended
T=000.00 Clay_Placement_S07_B02 to Cl

Moved:
  object:      Transport barge 02 contains: 0
  from:        5.09260 52.96039
  to:          5.91930 52.94240
Processed 1000:
  from:        Armour_Stock contains: 144000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.09260 52.96039
  to:          5.91930 52.94240
Processed 1000:
  from:        Armour_Stock contains: 143000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.91930 52.94240
  to:          5.06215 52.95060
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.91930 52.94240
  to:          5.06215 52.95060
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Armour_Layer_01 contains: 1000
Using Transport barge 02 to process 1000
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:

T=1458000.00 Start condition is satisfied, Levvel_Placement_S02_B02 transporting from Levvel_Stock to Levvel_Layer_01 started
Using Transport barge 02 to process 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Levvel_Layer_00 contains: 5000
T=1475891.47 Stop condition is satisfied, Levvel_Placement_S01_B02 transporting from Levvel_Stock to Levvel_Layer_00 complete
Moved:
  object:      Transport barge 02 contains: 0
  from:        5.04656 52.94579
  to:          5.01930 52.94240
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Clay_Stock contains: 125000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.07479 52.95726
  to:          5.91930 52.94240
Nothing to move
Nothing to move
Moved:
  object:      Transport bar

Nothing to move
Nothing to move
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.05920 52.95247
  to:          5.01930 52.94240
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Clay_Stock contains: 120000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.01930 52.94240
  to:          5.11789 52.97370
T=2285563.50 Stop condition is satisfied, Clay_Placement_S06_B02 transporting from Clay_Stock to Clay_Layer_05 complete
T=2286000.00 Start condition is satisfied, Clay_Placement_S07_B01 transporting from Clay_Stock to Clay_Layer_06 started
Using Transport barge 01 to process 1000
T=2286000.00 Start condition is satisfied, Clay_Placement_S07_B02 transporting from Clay_Stock to Clay_Layer_06 started
Nothing to move
T=2286000.00 Start

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Clay_Layer_07 contains: 2000
Using Transport barge 01 to process 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.14317 52.98701
  to:          5.91930 52.94240
Nothing to move
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.91930 52.94240
  to:          5.08449 52.96582
Nothing to move
Processed 1000:
  from:        Levvel_Stock contains: 133000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:       

...

Nothing to move
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Clay_Layer_15 contains: 5000
T=8882719.76 Stop condition is satisfied, Clay_Placement_S16_B02 transporting from Clay_Stock to Clay_Layer_15 complete
Using Transport barge 01 to process 1000
Moved:
  object:      Hopper contains: 1000
  from:        5.27142 52.96385
  to:          5.24166 53.04220
Processed 1000:
  from:        Hopper contains: 0
  by:          Hopper
  to:          Sand_Layer_15 contains: 2000
Using Hopper to process 1000
Moved:
  object:      Hopper contains: 0
  from:        5.24166 53.04220
  to:          5.27142 52.96385
Processed 1000:
  from:        Sand_Stock contains: 72000
  by:          Hopper
  to:          Hopper contains: 1000
Moved:
  object:      Hopper contains: 1000
  from:        5.27142 52.96385
  to:          5.24166 53.04220
Processed 1000:
  from:        Hopper contains: 0
  by:          Hopper
  to:          Sand_Layer_15

Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Clay_Stock contains: 63000
  by:          Gantry crane
  to:          Transport barge 01 contains: 1000
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.01930 52.94240
  to:          5.26959 53.05355
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 02 contains: 1000
  from:        5.91930 52.94240
  to:          5.21381 53.03055
Nothing to move
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Clay_Layer_17 contains: 2000
Using Transport barge 01 to process 1000
T=9835200.00 Start condition is satisfied, Armour_Placement_S15_B01 transporting from Armour_Stock to Armour_Layer_14 started
Using Transport barge 01 to process 1000
T=9835200.00 Start condition is satis

Nothing to move
Nothing to move
Moved:
  object:      Hopper contains: 1000
  from:        5.27142 52.96385
  to:          5.27958 53.06220
Processed 1000:
  from:        Hopper contains: 0
  by:          Hopper
  to:          Sand_Layer_18 contains: 1000
Using Hopper to process 1000
Nothing to move
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.22356 53.03926
  to:          5.91930 52.94240
Processed 1000:
  from:        Transport barge 02 contains: 0
  by:          Installation crane
  to:          Clay_Layer_18 contains: 5000
T=11070166.81 Stop condition is satisfied, Clay_Placement_S19_B02 transporting from Clay_Stock to Clay_Layer_18 complete
Moved:
  object:      Hopper contains: 0
  from:        5.27958 53.06220
  to:          5.27142 52.96385
Nothing to move
Processed 1000:
  from:        Sand_Stock contains: 58000
  by:          Hopper
  to:          Hopper contains: 1000
Nothing to move
Nothing to move
Moved:
  object:      Hopper contains: 1000
  from:

Moved:
  object:      Transport barge 01 contains: 1000
  from:        5.91930 52.94240
  to:          5.23620 53.04594
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Levvel_Layer_15 contains: 2000
Using Transport barge 01 to process 1000
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 02 contains: 0
  from:        5.23620 53.04594
  to:          5.91930 52.94240
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Armour_Stock contains: 65000
  by:          Gantry crane
  to:          Transport barge 02 contains: 1000
Nothing to move
Nothing to move
Nothing to move
Moved:
  object:      Transport barge 01 contains: 0
  from:        5.23620 53.04594
  to:          5.91930 52.94240
Nothing to move
Nothing to move
Nothing to move
Processed 1000:
  from:        Levvel_Stock contains: 

  to:          5.27413 53.06596
T=13593600.00 Start condition is satisfied, Levvel_Placement_S20_B01 transporting from Levvel_Stock to Levvel_Layer_19 started
Using Transport barge 01 to process 1000
T=13593600.00 Start condition is satisfied, Levvel_Placement_S20_B02 transporting from Levvel_Stock to Levvel_Layer_19 started
Using Transport barge 02 to process 1000
T=13593735.99 Stop condition is satisfied, Levvel_Placement_S19_B02 transporting from Levvel_Stock to Levvel_Layer_18 complete
Processed 1000:
  from:        Transport barge 01 contains: 0
  by:          Installation crane
  to:          Levvel_Layer_18 contains: 5000
T=13613527.23 Stop condition is satisfied, Levvel_Placement_S19_B01 transporting from Levvel_Stock to Levvel_Layer_18 complete
Moved:
  object:      Transport barge 02 contains: 0
  from:        5.27413 53.06596
  to:          5.91930 52.94240
Processed 1000:
  from:        Levvel_Stock contains: 54000
  by:          Gantry crane
  to:          Transport barge 

## Analysing the Results

In addition to the print statements provided by the Activity class, it also enters log entries into the logs of the origin, destination, mover, loader and unloader to mark when events such as loading or unloading the mover took place. For the mover, origin and destination, these log entries also contain the level of the container at the time of the log entry. This can be used for some basic visualisation of the simulation in Google Earth. 

In [12]:
# define an icon to use for the locations
icon = 'http://maps.google.com/mapfiles/kml/shapes/donut.png'
size = 1

# create a kml file containing the visualisation
kml = Kml()
fol = kml.newfolder(name="Dike Installation")

shared_style = Style()
shared_style.labelstyle.color = 'ffffffff'  # White
shared_style.labelstyle.scale = 1  
shared_style.iconstyle.color = 'ffff0000'  # Blue
shared_style.iconstyle.scale = 1
shared_style.iconstyle.icon.href = icon

# each stock should just be represented as a single point
for stock in stocks:
    pnt = fol.newpoint(name=stock.name, coords=[(stock.geometry.x, stock.geometry.y)])
    pnt.timestamp.when = my_env.epoch.isoformat()
    pnt.style = shared_style

for layer_sites in sites:
    for layer_site in layer_sites:
        # layer_site.value contains the container level for each log entry
        # layer_site.t contains the time of each log entry (seconds since the start of the simulation)
        # layer_site.log contains the message of each log entry
        
        # add points for each time the level increased
        coords = [(layer_site.geometry.x, layer_site.geometry.y)]
        for log_index, value in enumerate(layer_site.value[:-1]):
            # convert to real dates
            begin = my_env.epoch + datetime.timedelta(seconds=layer_site.t[log_index])
            end = my_env.epoch + datetime.timedelta(seconds=layer_site.t[log_index + 1])
            pnt = fol.newpoint(name='', coords=coords)
            pnt.timestamp.begin = begin.isoformat()
            pnt.timestamp.end = end.isoformat()
            
            # use custom style
            style = Style()
            style.labelstyle.color = 'ffffffff'  # White
            style.labelstyle.scale = 1  
            style.iconstyle.color = 'ffff0000'  # Blue
            style.iconstyle.scale = (value / layer_site.container.capacity) * size
            style.iconstyle.icon.href = icon
            pnt.style = style
        
        # add a final point for when the site has been completed
        begin = my_env.epoch + datetime.timedelta(seconds=layer_site.t[-1])
        end = my_env.epoch + datetime.timedelta(seconds=my_env.now)
        pnt = fol.newpoint(name='', coords=coords)
        pnt.timespan.begin = begin.isoformat()
        pnt.timespan.end = end.isoformat()
        
        # use custom style
        style = Style()
        style.labelstyle.color = 'ffffffff'  # White
        style.labelstyle.scale = 1  
        style.iconstyle.color = 'ff00ff00'   # Green
        style.iconstyle.scale = (layer_site.value[-1] / layer_site.container.capacity) * size
        style.iconstyle.icon.href = icon
        pnt.style = style

kml.save("sharedstyle.kml")

Opening the "sharedstyle.kml" file in Google Earth will show a simple animation of the layers and the order in which they are completed. 

In [13]:
# open the file
if platform.system():
    !start ./sharedstyle.kml
else:
    !start explorer ./sharedstyle.kml

Using the information in the log you could also create plots of the usage of each piece of equipment during the simulation. For example, the code below creates a plot of the usage of the two transporting barges.

In [14]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
class plot:
    """plot class"""

    def vessel_planning(vessels, activities, colors, web=False):
        """create a plot of the planning of vessels"""

        def get_segments(series, activity, y_val):
            """extract 'start' and 'stop' of activities from log"""
            x = []
            y = []
            for i, v in series.iteritems():
                if v == activity + ' start':
                    start = i
                if v == activity + ' stop':
                    x.extend((start, start, i, i, i))
                    y.extend((y_val, y_val, y_val, y_val, None))
            return x, y

        # organise logdata into 'dataframes' 
        dataframes = []
        for vessel in vessels:
            df = pd.DataFrame(
                {'log_value': vessel.value, 'log_string': vessel.log}, vessel.t)
            dataframes.append(df)
        df = dataframes[0]

        # prepare traces for each of the activities
        traces = []
        for i, activity in enumerate(activities):
            x_combined = []
            y_combined = []
            for k, df in enumerate(dataframes):
                y_val = vessels[k].name
                x, y = get_segments(
                    df['log_string'], activity=activity, y_val=y_val)
                x_combined.extend(x)
                y_combined.extend(y)
            traces.append(go.Scatter(
                name=activity,
                x=x_combined,
                y=y_combined,
                mode='lines',
                hoverinfo='y+name',
                line=dict(color=colors[i], width=10),
                connectgaps=False))
        
        # prepare layout of figure
        layout = go.Layout(
            title='Vessel planning',
            hovermode='closest',
            legend=dict(x=0, y=-.2, orientation="h"),
            xaxis=dict(
                title='Time',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'),
                range=[0, vessel.t[-1]]),
            yaxis=dict(
                title='Vessels',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f')))
        
        # plot figure
        init_notebook_mode(connected=True)        
        fig = go.Figure(data=traces, layout=layout)
        return iplot(fig, filename='news-source')

In [16]:
vessels = [transport_barge_01, transport_barge_02]
activities = ['loading', 'unloading', 'sailing full', 'sailing empty']
colors = {0:'rgb(55,126,184)', 1:'rgb(77,175,74)', 2:'rgb(255,0,0)', 3:'rgb(255,150,0)'}

plot.vessel_planning(vessels, activities, colors)